<p style="text-align: center;"> COST-EFFICIENT DEFENCE AGAINST UNMANNED AERIAL SYSTEMS </p>

---

In [1]:
import numpy as np
import numba as nb
import matplotlib.pyplot as plt

In [5]:
# TODO: laita githubiin ja jaa muille

n_sims = 100_000

asset_value = 300_000 # asset is for example an APC, tank, air defense system etc.
defense_budget_for_asset = 0.5 # pct of the assets value that can spent on its defense

threats = {
         # threat_name: {threat_price, [(distance, prob_of_destroying_asset),...], distribution_for_number_of_threats_in_a_single_attack}
         "FPV": {"price": 500, "prob": [(10_000, 0.1), (5_000, 0.2), (500, 0.4)], "dist": [0.1, 0.4, 0.3, 0.15, 0.05], "range": 5},
         "CMC": {"price": 3_000, "prob": [(10_000, 0.2), (5_000, 0.3), (500, 0.5)], "dist": [0.2, 0.5, 0.2, 0.1], "range": 4},
         "MMC": {"price": 20_000, "prob": [(10_000, 0.2), (5_000, 0.4), (500, 0.8)], "dist": [0.3, 0.5, 0.2], "range": 3}
        }

# each key in the dict is a name of the effector and each value is a set consisting of
# (price, Prob. of destroying threat 1, Prob. of destroying threat 2, ...)
effectors = {"shotgun": (300, 0.2, 0.1, 0.05), "gps_spoofer": (), "rf_jammer": (30_000, 0.9, 0.8, 0.6)}

In [3]:
def num_of_drones(drone_type: str) -> int:
    """
    Samples the number of a specific drone type in a single attack on the asset
    :drone_type (str): name of the drone type
    :return (int): the sampled number of drones
    """
    try:
        dist = threats[drone_type]["dist"]
        range = threats[drone_type]["range"]
        
        return np.random.choice(range, p=dist)
    
    except Exception as ex:
        raise (f"Failed to sample number of drones - {str(ex)}")

In [ ]:
def simulate_one_attack():
    pass

In [85]:
def simulate_n_attack(n: int):
    pass

3